# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weatherdata_df = pd.read_csv("weatherPy/output_data/cities.csv")
weatherdata_df.head(15)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yellowknife,62.46,-114.35,19.40,92,90,8.05,CA,1607302215
1,longyearbyen,78.22,15.64,28.40,86,90,19.46,SJ,1607302401
2,aldan,58.60,125.39,-0.69,91,100,12.41,RU,1607302401
3,ponta do sol,32.67,-17.10,59.00,82,40,5.82,PT,1607302401
4,georgetown,5.41,100.34,80.60,88,20,1.12,MY,1607302401
5,barrow,71.29,-156.79,1.40,70,1,8.05,US,1607302401
6,punta arenas,-53.15,-70.92,60.80,63,100,20.80,CL,1607302402
7,meyzieu,45.77,5.00,37.40,93,90,2.24,FR,1607302402
8,naziya,59.80,31.38,28.00,20,0,13.73,RU,1607302402
9,nadvoitsy,63.87,34.32,35.31,91,100,10.09,RU,1607302402


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# storing latitude and longitude in variable
locations = weatherdata_df[["Lat", "Lng"]]

# storing humidity as variable
humid = weatherdata_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Specific weather conditions for my ideal vacation spot: '>80 degrees', 'wind speed 5 or <', 'humidity < 50', 'cloudiness < 30'
conditions = weatherdata_df[(weatherdata_df["Max Temp"] < 80) | (weatherdata_df["Wind Speed"] > 10) | 
                            (weatherdata_df["Humidity"] > 60) | (weatherdata_df["Cloudiness"] > 30)].index
weatherdata_df.drop(conditions, inplace = True)
weatherdata_df

# Saving file for future reference
# weatherdata_df.to_csv("vacationPy/output_data/ideal_vacation_spots.csv")

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
349,kampot,10.62,104.18,80.60,57,0,6.93,KH,1607302448
363,general roca,-39.03,-67.58,86.00,27,0,6.93,AR,1607302450
367,karratha,-20.74,116.85,90.79,47,23,6.87,AU,1607302450


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [68]:
hotel_names = []
radius = 5000

# for index,row in weatherdata_df.iterrows():
#     coords = "\"%s\"" % f"{row['Lat']}, {row['Lng']}"

for row in range(len(weatherdata_df)):

    lats = weatherdata_df.iloc[row,1]
    lngs = weatherdata_df.iloc[row,2]
    coords = f"{lats},{lngs}"
    print(coords)

    params = {
        "location": coords,
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    response = requests.get(base_url, params=params)
    vacspots_hotel = response.json()


    try:
        hotel_names.append(vacspots_hotel["results"][row]["name"])
    except:
        hotel_names.append("Nearest hotel not found")

print(hotel_names)

10.62,104.18
-39.03,-67.58
-20.74,116.85
['Champa Lodge', 'Hotel Huemul', 'Cattrall Park Motel Karratha']


In [70]:
weatherdata_df["Hotel"] = hotel_names
hotel_df = weatherdata_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
349,kampot,10.62,104.18,80.60,57,0,6.93,KH,1607302448,Champa Lodge
363,general roca,-39.03,-67.58,86.00,27,0,6.93,AR,1607302450,Hotel Huemul
367,karratha,-20.74,116.85,90.79,47,23,6.87,AU,1607302450,Cattrall Park Motel Karratha


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
